<a href="https://colab.research.google.com/github/da-head0/mypetfood/blob/main/%EC%83%81%ED%92%88_%EA%B8%B0%EB%B0%98_%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/dataset/petfood.csv')
df

,category,brand,title,age,classification,content,nutrient,info,gram,calory,ingredient,detail,from_company,성분등록번호,img
0,고양이,지위픽 (ZIWI),주식캔 시식세트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://cdn.purplesto.re/media/store/sale/main...
1,고양이,뉴로 (NULO),뉴로 프리스타일 그레인프리 인도어 오리&렌틸콩,"키튼,어덜트,시니어,전연령",건사료,"오리, 닭고기, 칠면조, 대구, 완두콩, 렌틸콩, 닭고기지방, 콩섬유질, 천연향, ...",조단백 40% 조지방 17% 칼슘 1.1% 인 0.9% 수분 10%,"그레인프리,글루텐프리,비유전자변형,유산균",5.4kg,"3,724 kcal/kg","오리, 닭고기, 칠면조","연령구분 키튼,어덜트,시니어,전연령 푸드타입 건사료 중량 5.4kg 주재료 오리, ...",미국/Nulo Inc,서울-23093호,https://cdn.purplesto.re/media/store/sale/main...
2,고양이,게더 (GATHER),프리에이커 치킨 어덜트,"어덜트,시니어",건사료,"유기농 닭고기, 유기농 건조 닭고기, 유기농 완두콩, 렌틸콩, 닭지방, 유기농 아마...",조단백 29% 조지방 15% 칼슘 1.4% 인 0.6% 수분 10%,"그레인프리,글루텐프리,비유전자변형",1.8kg,"3,664 kcal/kg","유기농 닭, 유기농 완두콩, 렌즈콩","연령구분 어덜트,시니어 푸드타입 건사료 중량 1.8kg 주재료 유기농 닭, 유기농 ...",캐나다/Petcurean Pet Nutrition Inc.,제 서울-19594호,https://cdn.purplesto.re/media/store/sale/main...
3,개,윔지스 (WHIMZEES),밸류백 칫솔 L,"퍼피,어덜트,시니어,전연령",껌,"감자전분, 글리세린, 셀룰로오스 분말, 레시틴, 효모, 맥아 추출물, 루핀, 파프리...",조단백 1.1% 조지방 2.0% 조섬유 13.7% 조회분 2.4% 수분 12.0%,"그레인프리,글루텐프리,비유전자변형,채식",360g (6개입),"3,500 kcal/kg (210 kcal/개)","감자전분, 글리세린, 셀룰로오스 분말","연령구분 퍼피,어덜트,시니어,전연령 푸드타입 껌 중량 360g (6개입) 주재료 감...",네덜란드/Paragon Pet Products,제 서울-18050호,https://cdn.purplesto.re/media/store/sale/main...
4,개,아딜론덱 (ADIRONDACK),그레인프리 LID 흰살생선,"퍼피,어덜트,시니어,전연령",건사료,"흰살생선, 청어, 렌틸콩, 파바콩, 캐놀라오일, 타피오카, 비트펄프, 아마씨, 천연...",조단백 26% 조지방 15% 칼슘 1.2% 인 1% 수분 10%,"단일육류단백질,그레인프리,글루텐프리","5.4kg, 1.8kg, 11.3kg","3,571 kcal/kg","흰살생선, 청어, 렌틸콩","연령구분 퍼피,어덜트,시니어,전연령 푸드타입 건사료 중량 5.4kg, 1.8kg, ...","미국/Blackwood Pet Foods, Inc",NaN,https://cdn.purplesto.re/media/store/sale/main...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343,고양이,피니키,참치무스 캔,"키튼, 어덜트, 시니어",습식사료,"참치, 정제수, 타피오카전분, 참치 추출물, 해조류(진두발추출물), 구아검, 염화칼...","조단백 9.0%, 조지방 0.005%, 조섬유 1.00%, 조희분 3.00%, 칼슘...",수분함량높음,95g,NaN,"참치, 정제수, 타피오카전분",NaN,"태국/Unicord Public Co., Ltd",NaN,http://www.dogpia.net/agency/product_images/20...
1344,고양이,내추럴발란스,퍼펙트 파엘라 위드 흰살생선&정어리&케일 인 어 테이스티 그레이비,"키튼, 어덜트, 시니어",습식사료,"참치 육즙, 흰살생선, 정어리, 닭고기, 케일, 가리비, 건조계란, 타피오카 가루,...","조단백 10.0%, 조지방 2.0%, 조섬유 1.0%, 타우린 0.05%, 수분 8...",NaN,71g,820Kcal/100g,"참치 육즙, 흰살생선, 정어리",NaN,"태국/Natural Balance Pet Foods, Inc.",NaN,https://mamacat.co.kr/web/product/big/20191203...
1345,고양이,내추럴발란스,L.I.D. 닭고기 & 호박 수제 스튜 캣 포뮬라,NaN,습식사료,"닭고기, 닭고기 육즙, 호박, 카놀라 오일, 혼합 토코페롤 보존, 구아검, 인산칼슘...","조단백 10%, 조지방 1%, 조섬유 2%, 조회분 2%, 인 0.1%, 칼슘 0....",알러지예방식,78g,857Kcal/kg,"닭고기, 닭고기 육즙, 호박",NaN,"태국/Natural Balance Pet Foods, Inc.",NaN,https://www.naturalbalance.co.kr/wp-content/up...
1346,고양이,ANF,참치치킨무스캔 95g,"전연령, 키튼, 어덜트, 시니어",습식사료,"참치, 닭고기, 정제수, 젤리","조단백 9.0%, 조지방 0.4%, 조회분 3.0%, 조섬유 1.0%, 칼슘 0.0...",NaN,95g,NaN,"참치, 닭고기, 정제수",NaN,"태국/Unicord Public Co., Ltd",NaN,http://img.danawa.com/prod_img/500000/896/267/...


In [3]:
import numpy as np
import warnings
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

df = df[df.category == '고양이'] # 고양이만 남기기
print(df.shape)

(640, 15)


- 총 640개 데이터입니다.

In [7]:
df.isnull().sum()

category           0
brand              0
title              0
age               14
classification    15
content           15
nutrient          15
info              24
gram              15
calory            22
ingredient        15
detail            34
from_company      15
성분등록번호            95
img                0
dtype: int64

In [8]:
df[df.classification.isnull(]

,category,brand,title,age,classification,content,nutrient,info,gram,calory,ingredient,detail,from_company,성분등록번호,img
0,고양이,지위픽 (ZIWI),주식캔 시식세트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://cdn.purplesto.re/media/store/sale/main...
50,고양이,어플라우즈 (APPLAWS),토퍼시식세트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://cdn.purplesto.re/media/store/sale/main...
174,고양이,보레알 (BOREAL),주식캔 시식세트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://cdn.purplesto.re/media/store/sale/main...
330,고양이,지위픽 (ZIWI),에어드라이 시식세트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://cdn.purplesto.re/media/store/sale/main...
332,고양이,어플라우즈 (APPLAWS),통살 시식세트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://cdn.purplesto.re/media/store/sale/main...
652,고양이,오쥬바이로우즈 (AUJOU BY RAWZ),주식파우치 시식세트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://cdn.purplesto.re/media/store/sale/main...
768,고양이,어플라우즈 (APPLAWS),토퍼시식세트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://cdn.purplesto.re/media/store/sale/main...
875,고양이,K9 내추럴 (K9 NATURAL),피스트 주식캔 시식세트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://cdn.purplesto.re/media/store/sale/main...
905,고양이,어플라우즈 (APPLAWS),이지필시식세트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://cdn.purplesto.re/media/store/sale/main...
910,고양이,하이캣닢 (HIGH CATNIP),캣닢 드라이플라워 6g,"키튼,어덜트,시니어,전연령 색상 - 소재 건조된 캣닢 꽃 사이즈 - 중량/부피 6g...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"연령구분 키튼,어덜트,시니어,전연령 색상 - 소재 건조된 캣닢 꽃 사이즈 - 중량/...",NaN,NaN,https://cdn.purplesto.re/media/store/sale/main...


In [9]:
df.dropna(subset=['classification'], inplace=True) # 해당 열에 결측값이 있는 행 제거

In [15]:
df = df.reset_index(drop=True) # 인덱스 리셋

In [16]:
df_index = pd.Series(df.index, index = df.title)
df_index.head()

title
뉴로 프리스타일 그레인프리 인도어 오리&렌틸콩    0
프리에이커 치킨 어덜트                 1
에어드라이 닭고기                    2
주식파우치 no.15 닭고기와 꿩고기         3
파우치 닭고기와 아스파라거스              4
dtype: int64

In [ ]:
rating = pd.read_csv('/content/drive/MyDrive/dataset/cattaste.csv')
rating

In [ ]:
rating['사료명'] = rating['사료명'].replace('CAT','',regex=True)
rating['사료명'] = rating['사료명'].replace('DOG ','',regex=True)
rating['사료명'] = rating['사료명'].str.replace('\([^)]*\)','') # () 안 지시문 없애기
rating['사료명']  = rating['사료명'].replace(' \d.+g','',regex=True) # 성분등록번호 제거
rating['사료명']  = rating['사료명'].replace(' \d종','',regex=True) # 성분등록번호 제거
rating

In [31]:
rating = rating.rename(columns={"사료명": "title", "별점": "rating"})

In [32]:
rating.to_csv('/content/drive/MyDrive/dataset/catrating.csv', index=False)

In [34]:
# join

joined = df.merge(rating, how='left', on='title')
joined

,category,brand,title,age,classification,content,nutrient,info,gram,calory,ingredient,detail,from_company,성분등록번호,img,고양이_ID,고양이_이름,고양이_나이,rating
0,고양이,뉴로 (NULO),뉴로 프리스타일 그레인프리 인도어 오리&렌틸콩,"키튼,어덜트,시니어,전연령",건사료,"오리, 닭고기, 칠면조, 대구, 완두콩, 렌틸콩, 닭고기지방, 콩섬유질, 천연향, ...",조단백 40% 조지방 17% 칼슘 1.1% 인 0.9% 수분 10%,"그레인프리,글루텐프리,비유전자변형,유산균",5.4kg,"3,724 kcal/kg","오리, 닭고기, 칠면조","연령구분 키튼,어덜트,시니어,전연령 푸드타입 건사료 중량 5.4kg 주재료 오리, ...",미국/Nulo Inc,서울-23093호,https://cdn.purplesto.re/media/store/sale/main...,NaN,NaN,NaN,NaN
1,고양이,게더 (GATHER),프리에이커 치킨 어덜트,"어덜트,시니어",건사료,"유기농 닭고기, 유기농 건조 닭고기, 유기농 완두콩, 렌틸콩, 닭지방, 유기농 아마...",조단백 29% 조지방 15% 칼슘 1.4% 인 0.6% 수분 10%,"그레인프리,글루텐프리,비유전자변형",1.8kg,"3,664 kcal/kg","유기농 닭, 유기농 완두콩, 렌즈콩","연령구분 어덜트,시니어 푸드타입 건사료 중량 1.8kg 주재료 유기농 닭, 유기농 ...",캐나다/Petcurean Pet Nutrition Inc.,제 서울-19594호,https://cdn.purplesto.re/media/store/sale/main...,NaN,NaN,NaN,NaN
2,고양이,지위픽 (ZIWI),에어드라이 닭고기,"키튼,어덜트,시니어,전연령",저온건조 사료,"닭고기, 닭 내장(간, 심장) 및 뼈, 뉴질랜드 녹색홍합, 레시틴, 치커리 이눌린,...",조단백 44% 조지방 36% 칼슘 1.81% 인 1.38% 수분 8.0%,"그레인프리,글루텐프리",400g,"5,500 kcal/kg","닭고기, 닭 내장(간, 심장) 및 뼈, 뉴질랜드 녹색홍합","연령구분 키튼,어덜트,시니어,전연령 푸드타입 저온건조 사료 중량 400g 주재료 닭...",뉴질랜드/Ziwi Ltd,제 서울-15608호,https://cdn.purplesto.re/media/store/sale/main...,NaN,NaN,NaN,NaN
3,고양이,캣츠파인푸드 (CATZ FINEFOOD),주식파우치 no.15 닭고기와 꿩고기,"어덜트,시니어",습식사료,"닭, 물, 꿩, 쌀, 유채과, 미네랄, 민들레, 달맞이꽃 오일",조단백 8.6% 조지방 4% 칼슘 0.2% 인 0.15% 수분 88%,"검류미함유,비유전자변형",85g,965 kcal/kg,"닭, 물, 꿩","연령구분 어덜트,시니어 푸드타입 습식사료 중량 85g 주재료 닭, 물, 꿩 식단정보...",독일/Landguth Heimtiernahrung GmbH,GGHA50028호,https://cdn.purplesto.re/media/store/sale/main...,NaN,NaN,NaN,NaN
4,고양이,어플라우즈 (APPLAWS),파우치 닭고기와 아스파라거스,"키튼,어덜트,시니어,전연령",습식간식,"닭고기(가슴살), 닭육수, 아스파라거스, 쌀",조단백 14.0% 조지방 2.0% 조섬유 0.5% 조회분 1.0% 수분 82.0%,검류미함유,70g,532 kcal/kg (37.24 kcal/파우치),"닭고기(가슴살), 닭육수, 아스파라거스","연령구분 키튼,어덜트,시니어,전연령 푸드타입 습식간식 중량 70g 주재료 닭고기(가...",태국/MPM Products,제 서울-20530호,https://cdn.purplesto.re/media/store/sale/main...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,고양이,피니키,참치무스 캔,"키튼, 어덜트, 시니어",습식사료,"참치, 정제수, 타피오카전분, 참치 추출물, 해조류(진두발추출물), 구아검, 염화칼...","조단백 9.0%, 조지방 0.005%, 조섬유 1.00%, 조희분 3.00%, 칼슘...",수분함량높음,95g,NaN,"참치, 정제수, 타피오카전분",NaN,"태국/Unicord Public Co., Ltd",NaN,http://www.dogpia.net/agency/product_images/20...,1.0,미모,1.0,1.0
622,고양이,내추럴발란스,퍼펙트 파엘라 위드 흰살생선&정어리&케일 인 어 테이스티 그레이비,"키튼, 어덜트, 시니어",습식사료,"참치 육즙, 흰살생선, 정어리, 닭고기, 케일, 가리비, 건조계란, 타피오카 가루,...","조단백 10.0%, 조지방 2.0%, 조섬유 1.0%, 타우린 0.05%, 수분 8...",NaN,71g,820Kcal/100g,"참치 육즙, 흰살생선, 정어리",NaN,"태국/Natural Balance Pet Foods, Inc.",NaN,https://mamacat.co.kr/web/product/big/20191203...,1.0,미모,1.0,3.0
623,고양이,내추럴발란스,L.I.D. 닭고기 & 호박 수제 스튜 캣 포뮬라,NaN,습식사료,"닭고기, 닭고기 육즙, 호박, 카놀라 오일, 혼합 토코페롤 보존, 구아검, 인산칼슘...","조단백 10%, 조지방 1%, 조섬유 2%, 조회분 2%, 인 0.1%, 칼슘 0....",알러지예방식,78g,857Kcal/kg,"닭고기, 닭고기 육즙, 호박",NaN,"태국/Natural Balance Pet Foods, Inc.",NaN,https://www.naturalbalance.co.kr/wp-content/up...,1.0,미모,1.0,2.0
624,고양이,ANF,참치치킨무스캔 95g,"전연령, 키튼, 어덜트, 시니어",습식사료,"참치, 닭고기, 정제수, 젤리","조단백 9.0%, 조지방 0.4%, 조회분 3.0%, 조섬유 1.0%, 칼슘 0.0...",NaN,95g,NaN,"참치, 닭고기, 정제수",NaN,"태국/Unicord Public Co., Ltd",NaN,http://img.danawa.com/prod_img/500000/896/267/...,NaN,NaN,NaN,NaN


- ? 왜 inner지... 평가한 데이터가 있는 자료만 다루나?
- 나는 left로 바꿈

In [36]:
joined = joined[['고양이_ID', 'title', 'rating']]

pivot = pd.pivot_table(joined, index='title', columns='고양이_ID', values='rating')
pivot

고양이_ID,1.0,2.0,3.0
title,,,
그레인프리 치킨,NaN,1.0,NaN
L.I.D. 닭고기 & 호박 수제 스튜 캣 포뮬라,2.0,NaN,NaN
"내추럴 그레인프리 정어리, 새우&게살",1.0,NaN,NaN
내추럴 참치+치킨 캔,NaN,1.0,3.0
닭고기&닭간 페이트 포뮬라 캔니드 캣 푸드,NaN,2.0,NaN
라이프 에센셜 스프링클스 치킨,NaN,NaN,5.0
러브잇 다랑어흰살,5.0,NaN,NaN
러브잇 맛살,5.0,NaN,NaN
러브잇 연어&헤어볼,5.0,NaN,NaN


- 어차피 평가정보가 있는 것만 남는다..!!

In [37]:
# Center the mean around 0

pivot_norm = pivot.apply(lambda x: x - np.nanmean(x), axis=1)
pivot_norm

고양이_ID,1.0,2.0,3.0
title,,,
그레인프리 치킨,NaN,0.0,NaN
L.I.D. 닭고기 & 호박 수제 스튜 캣 포뮬라,0.0,NaN,NaN
"내추럴 그레인프리 정어리, 새우&게살",0.0,NaN,NaN
내추럴 참치+치킨 캔,NaN,-1.0,1.0
닭고기&닭간 페이트 포뮬라 캔니드 캣 푸드,NaN,0.0,NaN
라이프 에센셜 스프링클스 치킨,NaN,NaN,0.0
러브잇 다랑어흰살,0.0,NaN,NaN
러브잇 맛살,0.0,NaN,NaN
러브잇 연어&헤어볼,0.0,NaN,NaN


- 0점이... 되어버린다... 하지말자...

In [38]:
# fill Nan with 0
pivot.fillna(0, inplace=True)
pivot

고양이_ID,1.0,2.0,3.0
title,,,
그레인프리 치킨,0.0,1.0,0.0
L.I.D. 닭고기 & 호박 수제 스튜 캣 포뮬라,2.0,0.0,0.0
"내추럴 그레인프리 정어리, 새우&게살",1.0,0.0,0.0
내추럴 참치+치킨 캔,0.0,1.0,3.0
닭고기&닭간 페이트 포뮬라 캔니드 캣 푸드,0.0,2.0,0.0
라이프 에센셜 스프링클스 치킨,0.0,0.0,5.0
러브잇 다랑어흰살,5.0,0.0,0.0
러브잇 맛살,5.0,0.0,0.0
러브잇 연어&헤어볼,5.0,0.0,0.0


## Item Based Collaborative Filtering

In [39]:
# convert into dataframe
item_sim_df = pd.DataFrame(cosine_similarity(pivot, pivot), index=pivot.index, columns=pivot.index)
item_sim_df

title,그레인프리 치킨,L.I.D. 닭고기 & 호박 수제 스튜 캣 포뮬라,"내추럴 그레인프리 정어리, 새우&게살",내추럴 참치+치킨 캔,닭고기&닭간 페이트 포뮬라 캔니드 캣 푸드,라이프 에센셜 스프링클스 치킨,러브잇 다랑어흰살,러브잇 맛살,러브잇 연어&헤어볼,리얼 참치&새우,사이언스다이어트 어덜트 유리너리&헤어볼컨트롤,참치무스 캔,캡틴즈캐치 정어리&고등어,캣 헤어볼 케어,코어 그레인프리 헬시 인덜전스 머셀 닭고기와 닭간,코어 시그니쳐 셀렉트 – 슈레드 닭고기와 닭간,퍼펙트 파엘라 위드 흰살생선&정어리&케일 인 어 테이스티 그레이비,프리런 치킨 포 캣,화이트라벨 무스타입 주식캔 통치킨파이
title,,,,,,,,,,,,,,,,,,,
그레인프리 치킨,1.000000,0.0,0.0,0.316228,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.0,1.000000,1.000000
L.I.D. 닭고기 & 호박 수제 스튜 캣 포뮬라,0.000000,1.0,1.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,1.0,0.000000,0.000000
"내추럴 그레인프리 정어리, 새우&게살",0.000000,1.0,1.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,1.0,0.000000,0.000000
내추럴 참치+치킨 캔,0.316228,0.0,0.0,1.000000,0.316228,0.948683,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.316228,0.316228,0.0,0.316228,0.316228
닭고기&닭간 페이트 포뮬라 캔니드 캣 푸드,1.000000,0.0,0.0,0.316228,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.0,1.000000,1.000000
라이프 에센셜 스프링클스 치킨,0.000000,0.0,0.0,0.948683,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
러브잇 다랑어흰살,0.000000,1.0,1.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,1.0,0.000000,0.000000
러브잇 맛살,0.000000,1.0,1.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,1.0,0.000000,0.000000
러브잇 연어&헤어볼,0.000000,1.0,1.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,1.0,0.000000,0.000000


In [43]:
def get_similar_df(title):
  if title not in pivot.index:
    return None, None # 이거 고쳐야 할듯
  else:
    sim_dfs = item_sim_df.sort_values(by=title, ascending=False).index[1:]
    sim_score = item_sim_df.sort_values(by=title, ascending=False).loc[:, title].tolist()[1:]
    return sim_dfs, sim_score

In [44]:
dfs, score = get_similar_df("러브잇 맛살")
for x,y in zip(dfs[:10], score[:10]):
  print(f"{x} with similarity of {y}")

내추럴 그레인프리 정어리, 새우&게살 with similarity of 1.0
퍼펙트 파엘라 위드 흰살생선&정어리&케일 인 어 테이스티 그레이비 with similarity of 1.0
러브잇 다랑어흰살 with similarity of 1.0
러브잇 맛살 with similarity of 1.0
러브잇 연어&헤어볼 with similarity of 1.0
L.I.D. 닭고기 & 호박 수제 스튜 캣 포뮬라 with similarity of 1.0
사이언스다이어트 어덜트 유리너리&헤어볼컨트롤 with similarity of 1.0
참치무스 캔 with similarity of 1.0
캡틴즈캐치 정어리&고등어 with similarity of 1.0
캣 헤어볼 케어 with similarity of 1.0


In [54]:
# predict the rating of df x by cat y

def predict_rating(cat_id, title, max_neighbor=10):
    dfs, scores = get_similar_df(title)
    df_arr = np.array([x for x in dfs])
    sim_arr = np.array([x for x in scores])
    
    # select only the df that has already rated by user x
    filtering = pivot_norm[cat_id].loc[df_arr] != 0
    
    # calculate the predicted score
    s = np.dot(sim_arr[filtering][:max_neighbor], pivot[cat_id].loc[df_arr[filtering][:max_neighbor]]) \
            / np.sum(sim_arr[filtering][:max_neighbor])
    
    return s

In [61]:
predict_rating(1,'화이트라벨 무스타입 주식캔 통치킨파이')

0.0

In [63]:
predict_rating(2,'퍼펙트 파엘라 위드 흰살생선&정어리&케일 인 어 테이스티 그레이비')

0.0

- 대부분 0점으로 예측.... 근데 여기 최고점이 있나?? 

In [64]:
# recommend top n_df for user x based on item collaborative filtering algorithm
def get_recommendation(user_id, n_df=10):
    predicted_rating = np.array([])
    
    for _df in pivot_norm.index:
        predicted_rating = np.append(predicted_rating, predict_rating(user_id, _df))
    
    # don't recommend something that user has already rated
    temp = pd.DataFrame({'predicted':predicted_rating, 'name':pivot_norm.index})
    filtering = (pivot_norm[user_id] == 0.0)
    temp = temp.loc[filtering.values].sort_values(by='predicted', ascending=False)

    # recommend n_df df
    return df.loc[df_index.loc[temp.name[:n_df]]]

In [67]:
get_recommendation(3)

,category,brand,title,age,classification,content,nutrient,info,gram,calory,ingredient,detail,from_company,성분등록번호,img
294,고양이,캣만두 (CATMANDOO),라이프 에센셜 스프링클스 치킨,"키튼,어덜트,시니어,전연령",토퍼(부스터),닭고기 100%,조단백 74.0% 조지방 14.0% 조회분 7.0% 수분 6.0%,"그레인프리,글루텐프리",57g,"약 3,710 kcal/kg",닭고기,"연령구분 키튼,어덜트,시니어,전연령 푸드타입 토퍼(부스터) 중량 57g 주재료 닭고...",미국/Cat-man-doo,제 99QOD0001호,https://cdn.purplesto.re/media/store/sale/main...


- 아니 그냥 원래 좋다고 했는걸 뽑는데요...ㅎ